<a href="https://colab.research.google.com/github/retico/cmepda_medphys/blob/master/L8_code/Lecture8_demo_CAE_semantic_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DL methods for semantic segmentation

Semantic Segmentation is a classic Computer Vision problem which involves taking as input some raw data (e.g. 2D or 3D images) and converting them into a mask with regions of interest highlighted.

**Objective**

We will design and train a Convolutional Auto-Encoder (CAE) and then a U-net to segment mass lesions in mammograms.

We will use the dataset made available at:

https://drive.google.com/drive/folders/1gW2trBHf7Gw7zJKBYG32XP8mc2IbezdW?usp=drive_link

You can either add this folder to your drive ("Add shortcut to drive") or download the large_sample_Im_segmented_ref.zip we wil use, which contains 177 mass examples (104 benign and 73 malignant masses) and their segmentation masks.

**Legend of file names**

Mass lesions represented in each image.pgm are malignant if the file name ends with “_1.pgm”, benign if it ends with “_2.pgm”, e.g.:

0007p1_1_1.pgm contains a malignant mass

0003f1_1_1_2.pgm contains a benign mass


**Segmented masses.**

The folders whose name ends with "_ref", e.g.

small_sample_Im_segmented_ref/ and

large_sample_Im_segmented_ref/

contain for each original IMAGE or the small or large sample the IMAGE_resized.pgm and the IMAGE_mass_mask.pgm (which has the same size of the IMAGE_resized.pgm).
These lesion segmentation masks have been generated by using a simplified version of the semi-automated approach for mass lesion segmentation presented in the paper by Delogu *et al.*, Comput Biol Med (2007) [[ref]](https://pubmed.ncbi.nlm.nih.gov/17383623/).




## Reading data from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
!unzip -q /content/gdrive/MyDrive/CMEPDA_MedPhys_datasets/IMAGES/Mammography_masses/large_sample_Im_segmented_ref.zip -d /content/

In [ ]:
!ls /content/large_sample_Im_segmented_ref/ | head -n 4

In [ ]:
dataset_path = "/content/large_sample_Im_segmented_ref"

# Reading data from local directory

In [ ]:
#dataset_path = " ... local path to ... /DATASETS/IMAGES/Mammography_masses/large_sample_Im_segmented_ref"

##  Dataset overview

In [ ]:
import os
import PIL
from PIL import Image
import matplotlib.pyplot as plt

We have two kinds of images: *_resized*, i.e. the images containing the mass lesions, and *_mass_mask*, i.e. the lesion masks.

In [ ]:
os.path.join(dataset_path, "0069p1_4_2_resized.pgm")

In [ ]:
im_0 = Image.open(os.path.join(dataset_path, "0069p1_4_2_resized.pgm"))
im_1 = Image.open(os.path.join(dataset_path, "0069p1_4_2_mass_mask.pgm"))

In [ ]:
plt.imshow(im_0, cmap='bone')

In [ ]:
plt.imshow(im_1, cmap='jet', alpha = 0.5)

In [ ]:
plt.imshow(im_0, cmap='bone')
plt.imshow(im_1, cmap='jet', alpha = 0.5)

## Reading the images in memory

In [ ]:
import glob
import math
import numpy as np
from skimage.io import imread
from skimage.transform import resize

We can write a function to read alle images and masks as np.arrays and to return arrays with a shape suitable to train a DL model [i.e. A(n_img, size_x, size_y, 1)]. In additions, we rescale the image intensity in the [0,1]. To fasten the DL training phase, the images are resized to a smaller dimension, e.g. (64, 64).

In [ ]:
def read_dataset(dataset_path, x_id ="_resized", y_id="_mass_mask", res_W = 64, res_H = 64):
    fnames = glob.glob(os.path.join(dataset_path, f"*{x_id}.pgm"  ))
    X = []
    Y = []
    for fname in fnames:
        X.append(imread(fname)[:,:,np.newaxis])
        Y.append(imread(fname.replace(x_id, y_id))[:,:,np.newaxis])
    return resize(np.array(X), (np.array(X).shape[0],res_W,res_H,1), mode = 'reflect', order = 3 ), np.round(resize(np.array(Y), (np.array(X).shape[0],res_W,res_H,1), mode = 'reflect', order = 0)/255)


In [ ]:
X,Y = read_dataset(dataset_path)

In [ ]:
print(X.shape, Y.shape)
print(X.min(), X.max(), Y.min(), Y.max())

In [ ]:
plt.imshow(X[0].squeeze())
plt.show()

In [ ]:
plt.imshow(Y[0].squeeze())
plt.show()

# Train and test split

We split the dataset in a train and a test sets.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape)

We will use the *train set* (X_train) to train the model (allowing for an internal train-validation split). We will leave apart the *test set* (X_test) to evaluate the model performance.

# Defining and training a Convolutional Auto-Encoder (CAE)  model

We will design and train a convolutional autoencoder, inspired by the paper by (Liu et al,
*Deep convolutional neural network and 3D deformable approach for tissue segmentation in musculoskeletal Magnetic Resonance Imaging*, Magn Reson Med 2018; 79(4):2379-2391, https://onlinelibrary.wiley.com/doi/10.1002/mrm.26841 where we can find an example of a possible architecture.

In [ ]:
from keras.layers import Conv2D, Conv2DTranspose, Input
from keras.models import Model, load_model

In [ ]:
def make_model(shape=(64,64,1)):
    input_tensor = Input(shape=shape)
    x = Conv2D(32, (3,3), strides=2, padding='same', activation='relu')(input_tensor)
    x = Conv2D(64, (3,3), strides=2,  padding='same', activation='relu')(x)

    x = Conv2D(128, (3,3), strides=2, padding='same', activation='relu')(x)

    x = Conv2DTranspose(64, (3,3), strides=2, padding='same', activation='relu')(x)
    x = Conv2DTranspose(32, (3,3), strides=2, padding='same',activation='relu')(x)
    x = Conv2DTranspose(32, (3,3), strides=2, padding='same',activation='relu')(x)

    out = Conv2D(1, (1,1), padding='same',activation='sigmoid')(x)

    model = Model(input_tensor, out)

    return model

In [ ]:
model = make_model()
model.summary()

We compile the model

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

We can choose to save the model weights (save_best_only=True) that realized the best performance on the internal validation set (*early stop*) by means of a *callbacks*

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(
    "model_CAE.{epoch:02d}-{val_accuracy:.4f}.keras", # Changed .h5 to .keras
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch')

To fit the model we need to make an additional split of the *train* set into a *train* and a *validation* sets.

In [ ]:
history = model.fit(X_train,Y_train, validation_split=0.1, epochs=300, callbacks=[checkpoint])
# with these settings only after 50 epochs the validation accuracy starts to improves

We can visualize the loss and the accuracy on train and validation sets

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.figure()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['accuracy', 'val_accuracy'])

We can load a saved model, and evaluate its performance on the test examples

In [ ]:
!ls -ltr

In [ ]:
#Save a trained model in a specific folder:
#!cp model_CAE.101-0.9822.keras  /content/gdrive/My\ Drive/Colab\ Notebooks/CMEPDA_MedPhys/models_DL_segm/model_CAE.101-0.9822.keras

In [ ]:
!ls /content/gdrive/My\ Drive/Colab\ Notebooks/CMEPDA_MedPhys/models_DL_segm/

In [ ]:
# We can load one of the models we have just saved (the best performing one) or one of the models we have trained in a previous session and stored somewhere, e.g.:
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/CMEPDA_MedPhys/models_DL_segm/model_CAE.101-0.9822.keras .

model = load_model("model_CAE.101-0.9822.keras")

# We can check its architecture
model.summary()

We visualize the predicted segmentation on some images of the train set ...

In [ ]:
# we select one random example from the train set
idx = 11   # 14, 33, 98, 11
xtrain = X_train[idx][np.newaxis,...]
ytrain = Y_train[idx][np.newaxis,...]
print(xtrain.shape)

# and we plot the original image, the "ground truth mask" and the prediction of our model
plt.figure(figsize=(14,4))

ax1 = plt.subplot(1,3,1)
plt.imshow(xtrain.squeeze())
ax1.title.set_text('Original image')

ax2 = plt.subplot(1,3,2)
plt.imshow(ytrain.squeeze())
ax2.title.set_text('True Lesion Mask')

ax3 = plt.subplot(1,3,3)
plt.imshow(model.predict(xtrain).squeeze()>0.2) # You can remove the threshold ">0.2" and see the output
ax3.title.set_text('Predicted Lesion Mask')

and on images of the test set (never seen by the CAE)

In [ ]:
idx= 30 #  1, 15     18, 13
xtest = X_test[idx][np.newaxis,...]
ytest = Y_test[idx][np.newaxis,...]

plt.figure(figsize=(14,4))
plt.subplot(1,3,1)
plt.imshow(xtest.squeeze())
plt.subplot(1,3,2)
plt.imshow(ytest.squeeze())
plt.subplot(1,3,3)
plt.imshow(model.predict(xtest).squeeze()>0.2)

# Evaluation of the performance

We can quantify the segmentation performance on the train and test in terms of the Dice index

In [ ]:
def dice(pred, true, k = 1):
    intersection = np.sum(pred[true==k]) * 2.0
    dice = intersection / (np.sum(pred) + np.sum(true))
    return dice

We compute the Dice index for a single example, either of the train or test set:

In [ ]:
idx=67
y_pred = model.predict(X_train[idx][np.newaxis,...]).squeeze()>0.2
y_true = Y_train[idx].squeeze()

dice(y_pred, y_true)

We can also compute the Dice index for all images in the np.arrays of the train and test sets

In [ ]:
def dice_coef(pred, true, k = 1):
    intersection = 2.0 *np.sum(pred * (true==k), axis=(1,2,3))
    dice = intersection / (pred.sum(axis=(1,2,3)) + true.sum(axis=(1,2,3)))
    return dice

In [ ]:
Y_train.shape, X_train.shape

In [ ]:
dice_coef(Y_train,model.predict(X_train)>0.2)

In [ ]:
dice_coef(Y_test,model.predict(X_test)>0.2)

... and compute their mean values:

In [ ]:
dice_coef(Y_train,model.predict(X_train)>0.5).mean()

In [ ]:
dice_coef(Y_test,model.predict(X_test)>0.5).mean()

You can explore the dependence of the average Dice values on the threshold used to binarize the CAE output

# Defining and training a U-net  model

U-net models are extremely powerful techniques to segment medical images. They were introduced by Ronnemberger *et al.* with the paper *U-net: Convolutional networks for biomedical image segmentation*. Lect Notes Comput Sci 9351:234–41 (2015)  [[Ref]](https://doi.org/10.1007/978-3-319-24574-4_28).

There are a number of very informative tutorials on the web, e.g. [[Ref]](https://pyimagesearch.com/2022/02/21/u-net-image-segmentation-in-keras/). The following U-net implementation is derived from the [tutorial](https://github.com/bnsreenu/python_for_image_processing_APEER) by S. Bhattiprolu on this topic.

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate

It is practical to define functions for convolution, ancoding and decoding blocks:

In [ ]:
# Convolution block with 2 conv layers and batch normalization for each layer
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)   #Not in the original network.
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)  #Not in the original network
    x = Activation("relu")(x)
    return x

# Encoder block: Conv block followed by maxpooling.
# Returns both convolution and maxpooling outputs. The conv output can be used for concatenation (skip connections) with decoder.
def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

# Decoder Block: Skip features gets input from encoder for concatenation
def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

We can use these functions to build our model

In [ ]:
def build_unet(input_shape, n_classes):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024) #Bridge

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    if n_classes == 1:
      activation = 'sigmoid'
    else:
      activation = 'softmax'

    outputs = Conv2D(n_classes, 1, padding="same", activation=activation)(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model

This general formulation of the model allows to use it with input_shape=(256,256,3), similarly to the original implementation by  [[Ref]](https://github.com/bnsreenu/python_for_image_processing_APEER). It works also with an input_shape=(128,128,1) or (64,64,1)

In [ ]:
model_Unet = build_unet(input_shape=(64,64,1), n_classes=1)
print(model_Unet.summary())

This network has too many parameters, we can simplify the architecture a bit:

In [ ]:
def build_unet(input_shape, n_classes):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 8 )
    s2, p2 = encoder_block(p1, 16)
    s3, p3 = encoder_block(p2, 32)

    b1 = conv_block(p3, 64) #Bridge

    d1 = decoder_block(b1, s3, 32)
    d2 = decoder_block(d1, s2, 16)
    d3 = decoder_block(d2, s1, 8)

    if n_classes == 1:
      activation = 'sigmoid'
    else:
      activation = 'softmax'

    outputs = Conv2D(n_classes, 1, padding="same", activation=activation)(d3)

    model = Model(inputs, outputs, name="U-Net")
    return model

In [ ]:
model_Unet = build_unet(input_shape=(64, 64,1), n_classes=1)
print(model_Unet.summary())

In [ ]:
model_Unet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(
    "model_Unet.{epoch:02d}-{val_accuracy:.4f}.keras", # Changed .h5 to .keras
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='auto', save_freq='epoch')

In [ ]:
history = model_Unet.fit(X_train,Y_train, validation_split=0.1, epochs=300, callbacks=[checkpoint]) #

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.figure()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['accuracy', 'val_accuracy'])


In [ ]:
!ls -ltr

In [ ]:
#Save a trained model in a specific folder:
#!cp model_Unet.200-0.9878.keras /content/gdrive/My\ Drive/Colab\ Notebooks/CMEPDA_MedPhys/models_DL_segm/.

In [ ]:
!ls /content/gdrive/My\ Drive/Colab\ Notebooks/CMEPDA_MedPhys/models_DL_segm/

In [ ]:
# We can load one of the models we have just saved or one of the models we have trained in a previous session and stored somewhere, e.g.:
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/CMEPDA_MedPhys/models_DL_segm/model_Unet.200-0.9878.keras .

model_Unet = load_model("model_Unet.200-0.9878.keras")
model_Unet.summary()

We visualize the predicted segmentation on some images of the train set ...

In [ ]:
# we select one random example from the train set
idx = 11   # 11, 14, 33, 98
xtrain = X_train[idx][np.newaxis,...]
ytrain = Y_train[idx][np.newaxis,...]
print(xtrain.shape)

# and we plot the original image, the "ground truth mask" and the prediction of our model
plt.figure(figsize=(14,4))

ax1 = plt.subplot(1,3,1)
plt.imshow(xtrain.squeeze())
ax1.title.set_text('Original image')

ax2 = plt.subplot(1,3,2)
plt.imshow(ytrain.squeeze())
ax2.title.set_text('True Lesion Mask')

ax3 = plt.subplot(1,3,3)
plt.imshow(model_Unet.predict(xtrain).squeeze()>0.2) # You can remove the threshold ">0.2" and see the output
ax3.title.set_text('Predicted Lesion Mask')

and on images of the test set (never seen by the U-net)

In [ ]:
idx= 1 #  1, 15, 30     18, 13
xtest = X_test[idx][np.newaxis,...]
ytest = Y_test[idx][np.newaxis,...]

plt.figure(figsize=(14,4))
plt.subplot(1,3,1)
plt.imshow(xtest.squeeze())
plt.subplot(1,3,2)
plt.imshow(ytest.squeeze())
plt.subplot(1,3,3)
plt.imshow(model_Unet.predict(xtest).squeeze()>0.2)


# Evaluation of the performance

We can quantify the segmentation performance on the train and test set in terms of the Dice index:

In [ ]:
dice_coef(Y_train,model_Unet.predict(X_train)>0.5).mean()

In [ ]:
dice_coef(Y_test,model_Unet.predict(X_test)>0.5)

In [ ]:
dice_coef(Y_test,model_Unet.predict(X_test)>0.5).mean()

A fair comparison between the performances of the two DL models we built for image segmentation can be done once both have been fully optimizes. The optimization of the DL model architecture and of the training parameters requires a long trial and error phase devote to identify the model with a suitable complexity to face our segmentation problem and accounting for the constraints provided by the limited size of the available dataset.
